In [1]:
import numpy as np
import selfies as sf

In [2]:
chars=list(sf.get_semantic_robust_alphabet())
chars=sorted(chars)
chars=["[EOS]"]+chars

In [3]:
import bisect
bisect.bisect(chars,'[EOS]')
#chars[63]

52

In [4]:
import bisect
def find_idx(ch):
    if ch==chars[0]: return 0
    else: return bisect.bisect(chars,ch)-1
def encode_selfies(lst,one_hot=True,force_len=-1,pad=0,skip_lengthy=False):
    if type(lst)==str: lst=[lst]
    ret=[]
    for s in lst:
        s=list(sf.split_selfies(s))
        l=[]
        for t in s:
            l.append(find_idx(t))
        if force_len==-1: ret.append(np.array(l))
        else:
            ret.append(np.ones(force_len)*pad)
            x=np.array(l)
            try:
                ret[-1][:len(x)]=x
            except:
                if skip_lengthy: pass
                else: raise ValueError("".join(s),"is too long!")
        if one_hot:
            tret=[]
            for idx in ret[-1]:
                tret.append(np.zeros(len(chars)))
                tret[-1][int(idx)]=1.
            ret=ret[:-1]+[tret]
    return ret
def decode_selfies(lst,one_hot=True):
    if one_hot:
        lst=np.array(lst)
        lst=np.argmax(lst,axis=-1)
    ret=[]
    for s in lst:
        l=""
        for t in s:
            l+=chars[t]
        ret.append(l)
    return ret
def smiles_to_selfies(smilst,add_eos=False):
    ret=[]
    for smi in smilst:
        try:
            sfr = sf.encoder(smi)
            if add_eos: sfr+=chars[0]
            ret.append(sfr)
        except:
            ret.append(None)
    return ret

In [5]:
chk="[C][C][Branch1][EOS]"
decode_selfies(encode_selfies([chk]))[0]==chk # Should be True

True

In [6]:
smiles_to_selfies(["c1ccccc1","CC(C)C"],add_eos=True)

['[C][=C][C][=C][C][=C][Ring1][=Branch1][EOS]',
 '[C][C][Branch1][C][C][C][EOS]']